In [1]:
"""USAGE: %(program)s PATH_TO_MOVIELENS_1M_DIR
"""

from movielens_fetch import fetch_movielens
from flurs.recommender.fm import FMRecommender
from flurs.evaluator import Evaluator

import logging
import os
import sys
import pickle

try:

    f = open('evaluatosr.pkl', 'rb')
    evaluator = pickle.load(f)
    f.close()
except FileNotFoundError:
    program = os.path.basename(sys.argv[0])
    logger = logging.getLogger(program)

    logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s')
    logging.root.setLevel(level=logging.INFO)
    logger.info('running %s' % ' '.join(sys.argv))

    logging.info('converting data into FluRS input object')
    data = fetch_movielens(data_home='datasets/ml-1m', size='1m')
        

2018-05-26 23:15:59,642 : INFO : running /Users/sergio/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py -f /Users/sergio/Library/Jupyter/runtime/kernel-8f496720-8adc-4dd6-bf24-64c38e478e31.json
2018-05-26 23:15:59,643 : INFO : converting data into FluRS input object


new method 1


In [2]:

logging.info('initialize recommendation model and evaluation module')
rec = FMRecommender(p=sum(data.contexts.values()),  # number of dimensions of input vector
                    k=40,
                    l2_reg_w0=2.,
                    l2_reg_w=8.,
                    l2_reg_V=16.,
                    learn_rate=.004)
rec.initialize()
evaluator = Evaluator(rec, data.can_repeat)

n_batch_train = int(data.n_sample * 0.2)  # 20% for pre-training to avoid cold-start
n_batch_test = int(data.n_sample * 0.1)  # 10% for evaluation of pre-training
batch_tail = n_batch_train + n_batch_test

# pre-train
# 20% for batch training | 10% for batch evaluate
# after the batch training, 10% samples are used for incremental updating
logging.info('batch pre-training before streaming input')
evaluator.fit(
    data.samples[:n_batch_train],
    data.samples[n_batch_train:batch_tail],
    n_epoch=1  # single pass even for batch training
)

f = open('evaluator.pkl', 'wb')
pickle.dump(evaluator, f)
f.close()


2018-05-26 23:16:16,912 : INFO : initialize recommendation model and evaluation module
2018-05-26 23:16:16,913 : INFO : batch pre-training before streaming input


In [3]:
# 70% incremental evaluation and updating
logging.info('incrementally predict, evaluate and update the recommender')
res = evaluator.evaluate(data.samples[batch_tail:])

2018-05-26 23:21:09,551 : INFO : incrementally predict, evaluate and update the recommender


In [4]:
i = 0
for r in res:
    if i > 100:
        break
    print(r)
    i = i + 1

(0.3308330051201158, 301, 0.012059999999991078, 0.00017800000000534055)
(0.23269263486873792, 33, 0.009349000000000274, 0.00039999999999906777)
(0.1851410949762783, 1106, 0.014286000000012677, 0.0003079999999897609)
(0.34013719796284136, 847, 0.009551999999999339, 0.00017300000000375348)
(0.2906547457333951, 2189, 0.011150999999998135, 0.000686999999999216)
(0.2845992305001741, 1208, 0.006580999999997061, 0.00026599999999632473)
(0.28515887633334325, 1536, 0.007263999999992166, 0.00016599999999300508)
(0.3346769949929802, 48, 0.005969999999990705, 0.0001620000000031041)
(0.18745585886726235, 38, 0.007140000000006808, 0.00018799999999430383)
(0.29959908243660605, 2398, 0.006611000000006584, 0.0001469999999983429)
(0.1954952598470573, 1952, 0.007319999999992888, 0.00016300000000057935)
(0.34392855839559466, 461, 0.006285000000005425, 0.00014500000000339242)
(0.2872319074561185, 681, 0.006495999999998503, 0.0001560000000040418)
(0.3416651953281876, 1587, 0.006022999999999001, 0.0001370000

In [36]:
from flurs.data.entity import User, Item, Event
import numpy as np
user = User(20, np.zeros(0))
rec.recommend(user, np.array([x for x in range(0, 2645)]), [0 for x in range(0, 32)])

(array([1618, 1013,  879, ..., 1007,  826, 1368]),
 array([0.49948711, 0.53248541, 0.53503928, ..., 0.75408717, 0.75408717,
        0.75408717]))

In [22]:
data

{'can_repeat': False,
 'contexts': {'item': 18, 'others': 32, 'user': 0},
 'n_item': 3232,
 'n_sample': 226310,
 'n_user': 6014,
 'samples': [<flurs.data.entity.Event at 0x1512efacc0>,
  ...]}